1. get geo index for each county
2. write function to get DataFrame of 1. city 2. type 3. page token (optional)
3. write function to get 60 places DataFrame
4. get multiple city / types
5. join them
6. profit!

In [1]:
# Dependencies
import requests
import json
import time
import pandas as pd
import numpy as np
import pprint as pprint
# Google API Key
from config import gkey

def to_lat_lng(json):
  return '{0}, {1}'.format(json["lat"], json['lng'])

def get_cities_df(cities):
    points = []
    north_easts = []
    south_wests = []
    
    for target_city in cities: 
        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(target_city, gkey)
        # Run a request to endpoint and convert result to json
        response = requests.get(target_url).json()
        geometry = response["results"][0]["geometry"]

        # Extract latitude and longitude.
        points.append(to_lat_lng(geometry["location"]))
        north_easts.append(to_lat_lng(geometry["bounds"]["northeast"]))
        south_wests.append(to_lat_lng(geometry["bounds"]["southwest"]))

    return pd.DataFrame({
        "City": cities,
        "Point": points,
        "Northeast": north_easts,
        "Southwest": south_wests,
    }) 

# cities = ["Palo Alto, California"]
cities = ["San Francisco, California", "Redwood City, California", "Berkeley, California", "Palo Alto, California", "Mountain View, California", "Sunnyvale, California", "Cupertino, California", "Santa Clara, California", "San Jose, California", "Fremont, California"]
# cities_df = get_cities_df(cities)
# cities_df.to_csv('cities.csv')



In [2]:


def get_response(url, params):
    response = requests.get(url, params=params).json()
    if response['status'] == 'OK':
        return response
    else:
        print('Not OK response: ' + response['status'])
        return None

# Returns DataFrame
def request_places(city, location, type, gkey):    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # rankby:distance will result INVALID_REQUSET
    params = {
        "location": location,
        "radius": 10000, # hard-code distance
        "type": type,
        "key": gkey,
    }

    response = get_response(base_url, params)
    results= response['results']
    count = 1
    
    # If there is no page token, return result, otherwise concat result with next page.
    while (count < 3 and 'next_page_token' in response):
        count = count + 1
        token = response['next_page_token']
#         print(f'Getting next page: {count}, token: {token}')
        
        # Sleep a while so that token can be used.
        time.sleep(3)
        params = {
            "location": location,
            "radius": 10000, # hard-code distance
            "type": type,
            "key": gkey,
            "page_token": response['next_page_token'],
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = get_response(base_url, params)
        results.extend(response['results'])
    
    print(f'Total page count: {count}')
    return results

def to_data_frame(city, type, results):
    cities = []
    types = []
    place_ids = []
    names = []
    poi = []
    vicnities = []
#     locations = []
    lat = []
    lng = []
    
    for result in results:
        # Skip locality place, e.g. Palo Alto.
        if ('locality' in result['types']):
            continue

        cities.append(city)
        types.append(type)
        place_ids.append(result['place_id'])
        names.append(result['name'])
        poi.append(result['types'])
        vicnities.append(result['vicinity'])
#         locations.append(to_lat_lng(result['geometry']['location']))
        lat.append(result['geometry']['location']['lat'])
        lng.append(result['geometry']['location']['lng'])
    
    return pd.DataFrame({
        'City': cities,
        'Type': types,
        'Place ID': place_ids,
        'Name': names,
        'Point of interests': poi,
        'Address': vicnities,
#         'LatLng': locations,
        'Lat': lat,
        'Lng': lng,
    })

def get_places_data_frame(city, location, type, gkey):
    print(f'Getting plcaes, city: "{city}", location: "{location}", type: "{type}"');
    results = request_places(city, location, type, gkey)
    return to_data_frame(city, type, results) 

In [ ]:
cities_df = pd.read_csv("cities.csv")
type = 'supermarket'

def get_all_places_data_frame(type, gkey):
    frames = [];
    for index, row in cities_df.iterrows():
        frames.append(get_places_data_frame(row['City'], row['Point'], type, gkey))
    return pd.concat(frames)

place_df = get_all_places_data_frame(type, gkey)
place_df.to_csv(type + '.csv')
place_df





Getting plcaes, city: "San Francisco, California", location: "37.7749295, -122.4194155", type: "supermarket"
